In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ds200-2021-2022-car-prediction/sampleSolutions.csv
/kaggle/input/ds200-2021-2022-car-prediction/train.csv
/kaggle/input/ds200-2021-2022-car-prediction/test.csv


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 KB 12.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=1e3f5762ec48a9081004919b14aae6ada303accd3ed4484818831d25a0d50755
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5


In [3]:
import pandas as pd 

import pyspark

from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline



from pyspark.ml.feature import StringIndexer, VectorIndexer, StringIndexerModel, IndexToString
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/11 06:55:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Load and parse the data file, converting it to a DataFrame.
train = spark.read.format("csv").load("/kaggle/input/ds200-2021-2022-car-prediction/train.csv",header = 'True',inferSchema='True')
test = spark.read.format("csv").load("/kaggle/input/ds200-2021-2022-car-prediction/test.csv",header = 'True',inferSchema='True')

In [5]:
train.show(5)

+------+------------+-----------------+---------------+--------------+----------+------+-------------+
|car_id|buying_price|maintenance_price|number_of_doors|carry_capacity|trunk_size|safety|acceptability|
+------+------------+-----------------+---------------+--------------+----------+------+-------------+
| 10001|         low|             high|              4|             4|       med|   med|          acc|
| 10002|         med|              med|              2|             4|     small|   med|          acc|
| 10003|         low|            vhigh|              4|             2|       med|   low|        unacc|
| 10004|        high|             high|              2|          more|     small|   med|        unacc|
| 10005|         low|             high|              3|             2|       med|   low|        unacc|
+------+------------+-----------------+---------------+--------------+----------+------+-------------+
only showing top 5 rows



In [6]:
#convert string to index
labelIndexer = StringIndexer(inputCol="acceptability", outputCol="label") 
buyIndexer = StringIndexer(inputCol="buying_price", outputCol="indexedBuyPrice")

In [7]:
indexedLabelTrain=labelIndexer.fit(train).transform(train)

In [8]:
indexedLabelTrain.show(5)

+------+------------+-----------------+---------------+--------------+----------+------+-------------+-----+
|car_id|buying_price|maintenance_price|number_of_doors|carry_capacity|trunk_size|safety|acceptability|label|
+------+------------+-----------------+---------------+--------------+----------+------+-------------+-----+
| 10001|         low|             high|              4|             4|       med|   med|          acc|  1.0|
| 10002|         med|              med|              2|             4|     small|   med|          acc|  1.0|
| 10003|         low|            vhigh|              4|             2|       med|   low|        unacc|  0.0|
| 10004|        high|             high|              2|          more|     small|   med|        unacc|  0.0|
| 10005|         low|             high|              3|             2|       med|   low|        unacc|  0.0|
+------+------------+-----------------+---------------+--------------+----------+------+-------------+-----+
only showing top 5 

In [9]:
df1 = buyIndexer.fit(indexedLabelTrain).transform(indexedLabelTrain)
df1.show(2)

+------+------------+-----------------+---------------+--------------+----------+------+-------------+-----+---------------+
|car_id|buying_price|maintenance_price|number_of_doors|carry_capacity|trunk_size|safety|acceptability|label|indexedBuyPrice|
+------+------------+-----------------+---------------+--------------+----------+------+-------------+-----+---------------+
| 10001|         low|             high|              4|             4|       med|   med|          acc|  1.0|            2.0|
| 10002|         med|              med|              2|             4|     small|   med|          acc|  1.0|            1.0|
+------+------------+-----------------+---------------+--------------+----------+------+-------------+-----+---------------+
only showing top 2 rows



In [10]:
#decission_tree_classifier_model = DecisionTreeClassifier(labelCol="label", featuresCol="indexedBuyPrice",maxDepth=10)
#decission_tree_classifier_model.fit(df1)

In [11]:
assembler = VectorAssembler(inputCols=["indexedBuyPrice"],outputCol="features")
df2 = assembler.transform(df1)
df2.show(2)

+------+------------+-----------------+---------------+--------------+----------+------+-------------+-----+---------------+--------+
|car_id|buying_price|maintenance_price|number_of_doors|carry_capacity|trunk_size|safety|acceptability|label|indexedBuyPrice|features|
+------+------------+-----------------+---------------+--------------+----------+------+-------------+-----+---------------+--------+
| 10001|         low|             high|              4|             4|       med|   med|          acc|  1.0|            2.0|   [2.0]|
| 10002|         med|              med|              2|             4|     small|   med|          acc|  1.0|            1.0|   [1.0]|
+------+------------+-----------------+---------------+--------------+----------+------+-------------+-----+---------------+--------+
only showing top 2 rows



In [12]:
decission_tree_classifier_model2 = DecisionTreeClassifier(labelCol="label", featuresCol="features",maxDepth=10)
decission_tree_classifier_model2.fit(df2).transform(df2).show(5)

+------+------------+-----------------+---------------+--------------+----------+------+-------------+-----+---------------+--------+--------------------+--------------------+----------+
|car_id|buying_price|maintenance_price|number_of_doors|carry_capacity|trunk_size|safety|acceptability|label|indexedBuyPrice|features|       rawPrediction|         probability|prediction|
+------+------------+-----------------+---------------+--------------+----------+------+-------------+-----+---------------+--------+--------------------+--------------------+----------+
| 10001|         low|             high|              4|             4|       med|   med|          acc|  1.0|            2.0|   [2.0]|[1015.0,306.0,56....|[0.71078431372549...|       0.0|
| 10002|         med|              med|              2|             4|     small|   med|          acc|  1.0|            1.0|   [1.0]|[1015.0,306.0,56....|[0.71078431372549...|       0.0|
| 10003|         low|            vhigh|              4|          

In [13]:
pipeline = Pipeline( stages = [buyIndexer, assembler,decission_tree_classifier_model2, ])

In [14]:
model = pipeline.fit(indexedLabelTrain)
model.transform(indexedLabelTrain).show()


+------+------------+-----------------+---------------+--------------+----------+------+-------------+-----+---------------+--------+--------------------+--------------------+----------+
|car_id|buying_price|maintenance_price|number_of_doors|carry_capacity|trunk_size|safety|acceptability|label|indexedBuyPrice|features|       rawPrediction|         probability|prediction|
+------+------------+-----------------+---------------+--------------+----------+------+-------------+-----+---------------+--------+--------------------+--------------------+----------+
| 10001|         low|             high|              4|             4|       med|   med|          acc|  1.0|            2.0|   [2.0]|[1015.0,306.0,56....|[0.71078431372549...|       0.0|
| 10002|         med|              med|              2|             4|     small|   med|          acc|  1.0|            1.0|   [1.0]|[1015.0,306.0,56....|[0.71078431372549...|       0.0|
| 10003|         low|            vhigh|              4|          

In [15]:
testSolutions = pipeline.fit(indexedLabelTrain).transform(test).select('car_id','prediction')
testSolutions.show()

labelsArray = ["unacc","acc","good","vgood"]
testSolutions = IndexToString(inputCol="prediction", outputCol="acceptability", labels = labelsArray).transform(testSolutions)
testSolutions.show()


solutions = testSolutions.select('car_id','acceptability')
solutions.show()
solutions.toPandas().to_csv("dumpsolutions.csv",header=True, index=False)

+------+----------+
|car_id|prediction|
+------+----------+
| 11429|       0.0|
| 11430|       0.0|
| 11431|       0.0|
| 11432|       0.0|
| 11433|       0.0|
| 11434|       0.0|
| 11435|       0.0|
| 11436|       0.0|
| 11437|       0.0|
| 11438|       0.0|
| 11439|       0.0|
| 11440|       0.0|
| 11441|       0.0|
| 11442|       0.0|
| 11443|       0.0|
| 11444|       0.0|
| 11445|       0.0|
| 11446|       0.0|
| 11447|       0.0|
| 11448|       0.0|
+------+----------+
only showing top 20 rows

+------+----------+-------------+
|car_id|prediction|acceptability|
+------+----------+-------------+
| 11429|       0.0|        unacc|
| 11430|       0.0|        unacc|
| 11431|       0.0|        unacc|
| 11432|       0.0|        unacc|
| 11433|       0.0|        unacc|
| 11434|       0.0|        unacc|
| 11435|       0.0|        unacc|
| 11436|       0.0|        unacc|
| 11437|       0.0|        unacc|
| 11438|       0.0|        unacc|
| 11439|       0.0|        unacc|
| 11440|       0.0